# Basics - Pytorch Profiler

Best to run on colab if you are on a non CUDA device locally like mps.

Sources
- https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html


In [1]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
# Simple resnet
model = models.resnet18()

# Example input
inputs = torch.randn(5, 3, 224, 224)

PyTorch profiler is enabled through the context manager and accepts
a number of parameters, some of the most useful are:

- ``activities`` - a list of activities to profile:
   - ``ProfilerActivity.CPU`` - PyTorch operators, TorchScript functions and
     user-defined code labels (see ``record_function`` below);
   - ``ProfilerActivity.CUDA`` - on-device CUDA kernels;
   - Unfortunately they don't seem to have an MPS one (yet) as of 7dec2022
- ``record_shapes`` - whether to record shapes of the operator inputs;
- ``profile_memory`` - whether to report amount of memory consumed by
  model's Tensors;
- ``use_cuda`` - whether to measure execution time of CUDA kernels.

Note: when using CUDA, profiler also shows the runtime CUDA events
occuring on the host.

In [3]:
# Start profiling the CPU
# The record_function context manager a 'code range' that is being tracked
# You can create multiple of these 'ranges' which will be tracked in parralel
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:

    # record_function context manager to label arbitrary code ranges with user provided name
    # This means everything within that context is tracked as part of the name we give it 'model_inference'
    with record_function("model_inference"):
        model(inputs)

Note that we can use ``record_function`` context manager to label
arbitrary code ranges with user provided names
(``model_inference`` is used as a label in the example above).

Profiler allows one to check which operators were called during the
execution of a code range wrapped with a profiler context manager.
If multiple profiler ranges are active at the same time (e.g. in
parallel PyTorch threads), each profiling context manager tracks only
the operators of its corresponding range.
Profiler also automatically profiles the async tasks launched
with ``torch.jit._fork`` and (in case of a backward pass)
the backward pass operators launched with ``backward()`` call.

Let's print out the stats for the execution above:

In [4]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         8.23%      17.104ms        99.55%     206.778ms     206.778ms             1  
                     aten::conv2d         0.13%     260.000us        60.17%     124.992ms       6.250ms            20  
                aten::convolution         1.28%       2.662ms        60.05%     124.732ms       6.237ms            20  
               aten::_convolution         1.45%       3.012ms        58.77%     122.070ms       6.104ms            20  
         aten::mkldnn_convolution        57.18%     118.770ms        57.32%     119.058ms       5.953ms            20  
                 aten::batch_norm       

### Reading the profiled results
* Difference between ``Self CPU .. `` and ``CPU ..``
    * Operators in the model can call other 'child' operators. 
    * CPU: Includes the time spend in children operations
    * Self CPU: Excludes those, so only time spend on that specifci operator


Here we see that, as expected, most of the time is spent in convolution (and specifically in ``mkldnn_convolution``
for PyTorch compiled with MKL-DNN support).
Note the difference between self cpu time and cpu time - operators can call other operators, self cpu time excludes time
spent in children operator calls, while total cpu time includes it. You can choose to sort by the self cpu time by passing
``sort_by="self_cpu_time_total"`` into the ``table`` call.

To get a finer granularity of results and include operator input shapes, pass ``group_by_input_shape=True``
(note: this requires running the profiler with ``record_shapes=True``):

In [5]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         8.23%      17.104ms        99.55%     206.778ms     206.778ms             1                                                                                []  
                     aten::conv2d         0.09%     184.000us        34.51%      71.673ms      71.673ms             1                             [[5, 3, 224, 224], [64, 3, 7, 7], [], [], [], 

Note the occurence of ``aten::convolution`` twice with different input shapes.


### Now let's try to analyze GPU usage

Do note that we are also profiling CPU usage, so the output table is now relatively large and you'll need to scroll to the right to see the actual CUDA usage. 

Both are tracked, as some operations are performed on the GPU, while some do still need to happen on the CPU.

In [6]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.05%       3.196ms       100.00%        6.884s        6.884s       0.000us         0.00%       1.752ms       1.752ms             1  
                                           aten::conv2d         0.00%     115.000us        94.50%        6.505s     325.275ms       0.000us         0.00%       1.148ms      57.400us            20  
         

Note in my notebook: Note the occurence of on-device kernels in the output (e.g. ``sgemm_32x32x32_NN``).

A few things to note:
* CPU times seem to be really low, why is that?
    - Because most operations are now performed on CUDA, so the CPU does nothing for those operations
    - Please scroll to the right to see the actual CUDA output.
* What we do see is that ``aten:mkldnn_convolution`` has been changed in this model for ``aten:cudnn_convolution``. Which also takes most of the CUDA memory in this case.



### Using profiler to analyze memory consumption

PyTorch profiler can also show the amount of memory (used by the model's tensors)
that was allocated (or released) during the execution of the model's operators.
In the output below, 'self' memory corresponds to the memory allocated (released)
by the operator, excluding the children calls to the other operators.
To enable memory profiling functionality pass ``profile_memory=True``.

In [7]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

# profile_memory=True needs to be added to be able to analyze/profile memory usage
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

# Please scroll to the right to see memory usage
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.54%     503.000us         0.54%     503.000us       2.515us      94.86 Mb      94.86 Mb           200  
    aten::max_pool2d_with_indices        10.54%       9.837ms        10.54%       9.837ms       9.837ms      11.48 Mb      11.48 Mb             1  
                      aten::addmm         0.35%     324.000us         0.36%     337.000us     337.000us      19.53 Kb      19.53 Kb             1  
                       aten::mean         0.04%      39.000us         0.15%     143.000us     143.000us      10.

What do we see:
* As a reminder the ``Self``, means that any child operations are excluded from the total for that operation in the network
* We see that all the convolution layers don't seem to take any memory THEMSELVES

Now let's try to sort of the normal CPU memory, meaning the child operations are also included.

In [8]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.54%     503.000us         0.54%     503.000us       2.515us      94.86 Mb      94.86 Mb           200  
                 aten::batch_norm         0.09%      80.000us        12.24%      11.417ms     570.850us      47.41 Mb           0 b            20  
     aten::_batch_norm_impl_index         0.16%     147.000us        12.15%      11.337ms     566.850us      47.41 Mb           0 b            20  
          aten::native_batch_norm        11.66%      10.875ms        11.95%      11.152ms     557.600us      47.

What do we see:
* We see that the memory taken seems to stay about the same untill the ``atten:empty`` layer.
* This can indicate that the child the process of each operations do keep and move over some memory between layers which doesn't seem to be released that much as most layers have the same amount

### Using tracing functionality

Profiling results can be outputted as a .json trace file. After you run this, please download the trace.json file and then go to [chrome://tracing](chrome://tracing) and upload the file to see the visualization from it.


In [9]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    model(inputs)

prof.export_chrome_trace("trace.json")

What do we see:
* You see all the operations that happen in parralelel on a differrent row.
* For each operation you can click on it and inspect what operation happened, there like a simple add_ or perhaps a more complicated attention layer

### Examining stack traces

Profiler can be used to analyze Python and TorchScript stack traces.

Basically you see which files and functions/methods in those files are triggered for that specific operation.

This can be very usefull to figure out where a function or file is located that clocks up you model

In [10]:
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    with_stack=True,
) as prof:
    model(inputs)

# Print aggregated stats
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  Source Location                                            
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------  
                                aten::cudnn_convolution        41.66%       4.168ms        53.21%       5.324ms     266.200us       1.136ms        65.48%       1.136ms      56.800us            20  runpy.py(87): _run_code        

Note the two convolutions and the two callsites in ``torchvision/models/resnet.py`` script.

(Warning: stack tracing adds an extra profiling overhead.)


### Visualizing data as a flamegraph

Execution time (``self_cpu_time_total`` and ``self_cuda_time_total`` metrics) and stack traces
can also be visualized as a flame graph. To do this, first export the raw data using ``export_stacks`` (requires ``with_stack=True``):


In [11]:
prof.export_stacks("/tmp/profiler_stacks.txt", "self_cuda_time_total")

In [12]:
!git clone https://github.com/brendangregg/FlameGraph

Cloning into 'FlameGraph'...
remote: Enumerating objects: 1217, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1217 (delta 43), reused 59 (delta 32), pack-reused 1136
Receiving objects: 100% (1217/1217), 1.93 MiB | 907.00 KiB/s, done.
Resolving deltas: 100% (700/700), done.


In [13]:
!cd FlameGraph

In [14]:
!./flamegraph.pl --title "CUDA time" --countname "us." /tmp/profiler_stacks.txt > perf_viz.svg

/bin/bash: ./flamegraph.pl: No such file or directory


### Using profiler to analyze long-running jobs

PyTorch profiler offers an additional API to handle long-running jobs
(such as training loops). Tracing all of the execution can be
slow and result in very large trace files. To avoid this, use optional
arguments:

- ``schedule`` - specifies a function that takes an integer argument (step number)
  as an input and returns an action for the profiler, the best way to use this parameter
  is to use ``torch.profiler.schedule`` helper function that can generate a schedule for you;
- ``on_trace_ready`` - specifies a function that takes a reference to the profiler as
  an input and is called by the profiler each time the new trace is ready.

To illustrate how the API works, let's first consider the following example with
``torch.profiler.schedule`` helper function:

In [15]:
from torch.profiler import schedule

my_schedule = schedule(
    skip_first=10,
    wait=5,
    warmup=1,
    active=3,
    repeat=2)

Profiler assumes that the long-running job is composed of steps, numbered
starting from zero. The example above defines the following sequence of actions
for the profiler:

1. Parameter ``skip_first`` tells profiler that it should ignore the first 10 steps
   (default value of ``skip_first`` is zero);
2. After the first ``skip_first`` steps, profiler starts executing profiler cycles;
3. Each cycle consists of three phases:

   - idling (``wait=5`` steps), during this phase profiler is not active;
   - warming up (``warmup=1`` steps), during this phase profiler starts tracing, but
     the results are discarded; this phase is used to discard the samples obtained by
     the profiler at the beginning of the trace since they are usually skewed by an extra
     overhead;
   - active tracing (``active=3`` steps), during this phase profiler traces and records data;
4. An optional ``repeat`` parameter specifies an upper bound on the number of cycles.
   By default (zero value), profiler will execute cycles as long as the job runs.

Thus, in the example above, profiler will skip the first 15 steps, spend the next step on the warm up,
actively record the next 3 steps, skip another 5 steps, spend the next step on the warm up, actively
record another 3 steps. Since the ``repeat=2`` parameter value is specified, the profiler will stop
the recording after the first two cycles.

At the end of each cycle profiler calls the specified ``on_trace_ready`` function and passes itself as
an argument. This function is used to process the new trace - either by obtaining the table output or
by saving the output on disk as a trace file.

To send the signal to the profiler that the next step has started, call ``prof.step()`` function.
The current profiler step is stored in ``prof.step_num``.

The following example shows how to use all of the concepts above:


In [16]:
# This is called at the end of each cycle we specified
# It's used to process a new trace by obtaining new table ourput OR saving output on disk as a trace file
def trace_handler(p):
    output = p.key_averages().table(sort_by="self_cuda_time_total", row_limit=10)
    print(output)
    p.export_chrome_trace("/tmp/trace_" + str(p.step_num) + ".json")

# Schedule to use
schedule = torch.profiler.schedule(
    wait=1,
    warmup=1,
    active=2
)


with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=schedule,
    on_trace_ready=trace_handler) as p:

    # Run an example for 8 steps
    for idx in range(8):
        model(inputs)

        # profiler.step sends a signal to the profiler that next step has started
        # Current step is stored as profile.step_num
        p.step()

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                aten::cudnn_convolution        27.78%       4.110ms        36.43%       5.389ms     134.725us       2.267ms        65.43%       2.267ms      56.675us            40  
void cudnn::ops::nchwToNhwcKernel<float, float, floa...         0.00%       0.000us         0.00%       0.000us       0.000us     652.000us        18.82%     652.000us       9.588us            68  
         

What we see:
* It only outputted 2 times, as the trace_handler is only called twice due to the schedule being used on only 8 iterations